MIT License

Copyright (c) 2024 Justin Randall, Smart Interactive Transformations Inc.

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Overview

The purpose of this notebook is to serve as basic documentation for using the Interactigen library to generate base and augmented utterance phrases to use with natural language understanding (NLU) intent classification, or with Large Language Model (LLM) fine-tuning.

The Interactigen module is built on LangChain.  The decision to use LangChain enables experimenting with various models with minimal code adjustments.  Already have code to load and configure your favourite LangChain ChatModel?  Great!  Let's use that.

The output results for each run is a JSON array for downstream use: write to a file, split test/train utterances, and more.

## Generate Phrase Utterances with OpenAI

Here is are two examples loading the OpenAI gpt-4-0125-preview and gpt-3.5-turbo-0125 models for generating the utterances.

Make sure you have an environment file with the path *scratch/dev.env* and set the OPENAI_API_KEY and (optionally) LANGCHAIN_API_KEY and LANGCHAIN_PROJECT if you are tracing with LangSmith:
```scratch/dev.env:
OPENAI_API_KEY=<OpenAI API Key>
LANGCHAIN_API_KEY=<LangSmith API Key>
LANGCHAIN_PROJECT=<LangSmith Project Name>
```

In [5]:
#! pip install dotenv langchain_openai
from interactigen import Interactigen
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(dotenv_path='../scratch/dev.env')

model = ChatOpenAI(model="gpt-4-0125-preview")

interactigen = Interactigen(
    model=model,
)

phrase_utterances = interactigen.generate_phrase_utterances(
    base_phrase="someone might express appreciation to a bot",
    init_quantity=20,
    lc_project='20240229-gen-phrasing-r1-openai-gpt-4-0125-preview'
)

phrase_utterances

['Thanks for your help!',
 'I appreciate your assistance.',
 "You've been really helpful, thank you.",
 'Much appreciated!',
 'Thank you, that was exactly what I needed.',
 'Cheers for the help!',
 "I'm so grateful for your assistance.",
 'Your support means a lot, thanks.',
 "Thank you, you've been great.",
 'Thanks a ton!',
 "You're a lifesaver, thank you.",
 "I can't thank you enough for this.",
 'Your help has been invaluable, thank you.',
 'Big thanks for your quick response!',
 'I owe you one, thanks!',
 'Your guidance was perfect, thanks!',
 'Thank you for being so helpful!',
 'You did a fantastic job, thank you.',
 'Thanks for going above and beyond!',
 'Your efforts are greatly appreciated!',
 'Thanks!',
 'Appreciate it.',
 'Really helpful, thanks.',
 'Much thanks!',
 'Exactly what I needed, thanks.',
 'Cheers!',
 'So grateful, thanks.',
 'Thanks a lot!',
 "You've been great, thanks.",
 'Thanks a ton!',
 'Lifesaver, thanks.',
 "Can't thank you enough.",
 'Invaluable help, than

In [6]:
#! pip install dotenv langchain_openai
from interactigen import Interactigen
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(dotenv_path='../scratch/dev.env')

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

interactigen = Interactigen(
    model=model,
)

phrase_utterances = interactigen.generate_phrase_utterances(
    base_phrase="someone might express appreciation to a bot",
    init_quantity=20,
    lc_project='20240229-gen-phrasing-r1-openai-gpt-3.5-turbo-0125'
)

phrase_utterances

['Thank you so much!',
 'I appreciate your help!',
 'Thanks a lot for assisting me!',
 'Your support is invaluable!',
 'Grateful for your guidance!',
 'Much obliged for your assistance!',
 'Thanks a bunch!',
 "I'm thankful for your help!",
 'Kudos to you for your support!',
 'Big thank you for your guidance!',
 'Thanks for being so helpful!',
 'Appreciate your prompt assistance!',
 'Thank you for being so responsive!',
 'Your help is greatly appreciated!',
 'Thanks for always being there!',
 "I'm grateful for your expertise!",
 'Thank you for your valuable input!',
 'Appreciate all your efforts!',
 'Thanks for going above and beyond!',
 'Your assistance means a lot to me!',
 'Thanks a lot!',
 'Appreciate your help!',
 'Big thank you!',
 'Your support rocks!',
 'Thanks for your guidance!',
 'Much obliged!',
 'Thanks!',
 'Thanks for helping!',
 'Kudos to you!',
 'Big thanks!',
 'Thanks for helping!',
 'Thanks for assisting!',
 'Thank you for being responsive!',
 'Greatly appreciated!',
 

## Generate Phrase Utterances with Google Generative AI

Here is an example loading the Google Generative AI gemini-pro model for generating the utterances.

Make sure you have an environment file with the path *scratch/dev.env* and set the GOOGLE_API_KEY and (optionally) LANGCHAIN_API_KEY and LANGCHAIN_PROJECT if you are tracing with LangSmith:
```scratch/dev.env:
GOOGLE_API_KEY=<Google Generative AI API Key>
LANGCHAIN_API_KEY=<LangSmith API Key>
LANGCHAIN_PROJECT=<LangSmith Project Name>
```

In [7]:
#! pip install dotenv langchain_google_genai
from interactigen import Interactigen
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv(dotenv_path='../scratch/dev.env')

model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

interactigen = Interactigen(
    model=model,
)

phrase_utterances = interactigen.generate_phrase_utterances(
    base_phrase="someone might express appreciation to a bot",
    init_quantity=20,
    lc_project='20240229-gen-phrasing-r1-google-gemini-pro'
)

phrase_utterances

['Thank you so much for your help, I really appreciate it!',
 "I'm so grateful for your assistance.",
 "You've been such a great help, I can't thank you enough.",
 "Your help has been invaluable, I'm very appreciative.",
 "I'm so glad I reached out to you, you've been so helpful.",
 'Thank you for your time and patience, I appreciate your help very much.',
 'I really appreciate you taking the time to help me.',
 'Your help has been essential in solving my problem, thank you so much.',
 "I'm so grateful for your assistance, thank you for going above and beyond.",
 "You've been a lifesaver, I'm so appreciative of your help.",
 "I'm very impressed with your knowledge and helpfulness, thank you.",
 "You've been so patient and understanding, I really appreciate it.",
 "I'm so glad I contacted you, your help has been invaluable.",
 "Thank you for your help, I couldn't have done it without you.",
 "I'm so grateful for your assistance, you've made a big difference.",
 "You've been so helpful, 